# validation data 합치기(macc, m_usage_stats 빼고)

## val 데이터 불러오기


In [ ]:
import os
import pandas as pd
import numpy as np
# 데이터 세트의 저장 위치 설정
challenge2024_dataset_path = "/Users/aswm0121/Downloads/휴먼이해2024"
challenge2024_dataset_path1 = "/Users/aswm0121/Downloads/휴먼이해2024/val dataset"
challenge2024_dataset_path2 = "/Users/aswm0121/Downloads/휴먼이해2024/test dataset"


## mActivity
1일 평균을 구하는 대신 **하루동안 각 활동을 몇 번 했는지 카운팅**해주었다.
결과적으로 **105 row의 dataframe으로 합치는 데 성공**
> 스마트폰에서 인식된 행동 분류값. **1분마다 1회씩** 기록됨.
  
- subject_id: 실험 참여자의 식별자
- timestamp
- m_activity
- 0: IN_VEHICLE
- 1: ON_BICYCLE
- 2: ON_FOOT
- 3: STILL
- 4: UNKNOWN
- 5: TILTING
- 7: WALKING
- 8: RUNNING
  

  

In [ ]:
# 파일 불러오기
file_name = "ch2024_val__m_activity.parquet.gzip"
df_mact = pd.read_parquet(os.path.join(challenge2024_dataset_path1, file_name))


# 년 월 달 의 정보만 가지고 있는 컬럼 생성(날짜별로 묶어주기 위해)
df_mact['Date'] = pd.to_datetime(df_mact['timestamp'], format='%Y-%m-%d').dt.date

# 기존의 timestamp 컬럼 삭제한다
df_mact.drop(columns = 'timestamp', axis = 1, inplace = True)
#df_mact

# subject_id, Date로 묶어서 활동을 총 몇 번 했는지 알려주는 컬럼 생성
df_new = df_mact.groupby(['subject_id', 'Date'])['m_activity'].count().reset_index()
#df_new

# subject_id, Date로 묶어준다음 0~8번 활동을 하루에 각각 몇 번씩 했는지 알기 위해 groupby 사용
activity_count = df_mact.groupby(['subject_id', 'Date'])['m_activity'].value_counts().unstack(fill_value=0)
# activity_count

# 컬럼을 0~8 대신에 숫자에 해당하는 활동명으로 바꿔줌
activity_count.columns = ['IN_VEHICLE', 'ON_BICYCLE', 'STILL', 'UNKNOWN','WALKING', 'RUNNING']

print("Activity counts:")
#activity_count

# 총 활동 횟수 df + 하루에 각 활동을 몇 번 했는지 알려주는 df
df_mact_fin = df_new.merge(activity_count, on=['subject_id', 'Date'], how='left')
df_mact_fin

Activity counts:


,subject_id,Date,m_activity,IN_VEHICLE,ON_BICYCLE,STILL,UNKNOWN,WALKING,RUNNING
0,1,2023-08-20,1440,105,17,1153,0,150,15
1,1,2023-08-21,1440,92,25,1061,0,256,6
2,1,2023-08-22,1440,105,48,869,0,402,16
3,1,2023-08-23,1440,133,51,963,0,293,0
4,1,2023-08-24,1440,141,10,1069,0,215,5
...,...,...,...,...,...,...,...,...,...
100,4,2023-10-27,1431,73,0,1336,1,21,0
101,4,2023-10-28,1440,47,0,1385,0,8,0
102,4,2023-10-29,1440,121,0,1303,0,16,0
103,4,2023-10-30,1440,60,0,1369,0,11,0


## mAmbience
> **105 rows × 273 columns로 합치는 것 성공**
- 스마트폰에서 인식된 음향 기반 레이블. 2분마다 1회씩 기록됨.
- subject_id: 실험 참여자의 식별자
- timestamp
- ambience_labels: **상위 10개의 레이블 및 각각의 확률 목록(해당하는 소리일 확률)**

<예시>
![image.png](attachment:ef5ddc0e-774f-429e-a0e6-9b7d31c5c3a5.png)
- 예시를 보면 첫 번째 리스트의 사운드 종류일 확률이 0.78로 제일 높다



In [ ]:
file_name = "ch2024_val__m_ambience.parquet.gzip"
df_mamb = pd.read_parquet(os.path.join(challenge2024_dataset_path1, file_name))
df_mamb.info()


a = df_mamb['ambience_labels']
lists =[]
for i in a: # 제일 확률이 높은 사운드만 따로 리스트에 넣어줌
    if (i.size == 0): # 해당 컬럼에 비어있는 리스트도 있어서 이때는 값을 unknown으로 해줌
        lists.append('unknown')
    else:
        lists.append(i[0][0])

#lists

df_mamb['sound_variety'] = lists
new_df_mamb = df_mamb.copy()

new_df_mamb.drop(columns = 'ambience_labels', axis = 1, inplace = True)
# new_df_mamb

# 컬럼에 들어갈 컬럼명들을 중복을 제거하고 정렬하여 넣어줌
unique_list = set(lists)
unique_list = sorted(unique_list) # 271개의 서로 다른 사운드 종류가 읶다


# 기존의 timestamp 컬럼 삭제한다
new_df_mamb['Date'] = pd.to_datetime(df_mamb['timestamp'], format='%Y-%m-%d').dt.date
new_df_mamb.drop(columns = 'timestamp', axis = 1, inplace = True)
#new_df_mamb



sound_count = new_df_mamb.groupby(['subject_id', 'Date'])['sound_variety'].value_counts().unstack(fill_value=0)
# sound_count


sound_count.columns = unique_list
print("sound_counts:")
sound_count = sound_count.reset_index()
sound_count

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74575 entries, 0 to 74574
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   subject_id       74575 non-null  int64         
 1   timestamp        74575 non-null  datetime64[us]
 2   ambience_labels  74575 non-null  object        
dtypes: datetime64[us](1), int64(1), object(1)
memory usage: 1.7+ MB
sound_counts:


,subject_id,Date,"Accelerating, revving, vroom",Air conditioning,"Air horn, truck horn",Aircraft,Alarm,Alarm clock,Animal,Applause,...,White noise,"Whoosh, swoosh, swish",Wild animals,Wind,Wind chime,Wind noise (microphone),Wood,Writing,Zipper (clothing),unknown
0,1,2023-08-20,0,0,0,0,0,0,2,0,...,6,0,0,2,0,0,1,1,0,0
1,1,2023-08-21,0,0,0,1,0,0,6,0,...,41,0,0,5,0,0,0,0,1,0
2,1,2023-08-22,0,0,0,0,0,0,10,0,...,30,0,1,0,0,0,0,0,0,0
3,1,2023-08-23,0,0,0,1,0,0,15,0,...,6,0,0,3,0,0,0,0,0,0
4,1,2023-08-24,0,0,0,0,0,0,14,0,...,9,0,0,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,4,2023-10-27,0,0,0,0,0,0,28,0,...,4,0,1,0,0,0,0,0,0,0
101,4,2023-10-28,0,0,0,0,0,0,25,0,...,9,0,0,5,0,0,1,1,0,0
102,4,2023-10-29,0,0,0,0,0,0,29,1,...,17,0,1,1,0,1,0,0,0,0
103,4,2023-10-30,0,0,0,1,1,0,27,0,...,0,0,0,2,0,0,0,0,0,0


## mGps
스마트폰에서 산출된 GPS 좌표 정보(단, 위도 및 경도는 상대 좌표로 변환됨). 5초 간격(1분당 약 12회)으로 측정됨.
> ** 이상한 건 105row가 아니라 101row란 점**. 어떤 날짜가 빠졌는지 확인해봐야 함
- subject_id: 실험 참여자의 식별자
- timestamp
- altitude: 고도
- latitude: 위도
- longitude: 경도
- speed: 속도

In [ ]:
# 데이터 불러오기
file_name = "ch2024_val__m_gps.parquet.gzip"
df_mgps = pd.read_parquet(os.path.join(challenge2024_dataset_path1, file_name))

# Date 컬럼 새로 만들고 기존의 timestamp 컬럼 제거
df_mgps['Date'] = pd.to_datetime(df_mgps['timestamp'], format='%Y-%m-%d').dt.date
df_mgps.drop(columns = 'timestamp', axis = 1, inplace = True)

# subject_id, Date로 묶어주고 고도,위도, 경도, 속도의 하루치 평균을 구해준다
df_mgps_new = df_mgps.groupby(['subject_id', 'Date'])[['altitude', 'latitude', 'longitude', 'speed']].mean().reset_index()
df_mgps_new

,subject_id,Date,altitude,latitude,longitude,speed
0,1,2023-08-20,133.974983,0.013321,0.930964,0.240203
1,1,2023-08-21,134.826176,0.013257,0.930848,0.178509
2,1,2023-08-22,125.524925,0.013310,0.930718,0.195090
3,1,2023-08-23,151.478346,0.013566,0.931541,0.274662
4,1,2023-08-24,128.542758,0.013114,0.931089,0.136859
...,...,...,...,...,...,...
96,4,2023-10-27,150.200178,0.498684,0.216652,0.332864
97,4,2023-10-28,146.588570,0.497337,0.222417,0.252733
98,4,2023-10-29,148.328821,0.501790,0.214975,0.401876
99,4,2023-10-30,145.996744,0.501440,0.213538,0.750531


## mLight(완)
스마트폰에서 측정된 빛의 세기. 10분 간격으로 측정됨.
> **106 rows로 이상한 한 개가 추가되었다..**
- subject_id: 실험 참여자의 식별자
- timestamp
- m_light: 빛의 세기

In [ ]:
# 데이터 불러오기
file_name = "ch2024_val__m_light.parquet.gzip"
df_mlight = pd.read_parquet(os.path.join(challenge2024_dataset_path1, file_name))

# Date 컬럼 새로 만들고 기존의 timestamp 컬럼 제거
df_mlight['Date'] = pd.to_datetime(df_mlight['timestamp'], format='%Y-%m-%d').dt.date
df_mlight.drop(columns = 'timestamp', axis = 1, inplace = True)


# subject_id, Date로 묶어주고 m_light의 하루치  평균을 구해준다
df_mlight_new = df_mlight.groupby(['subject_id', 'Date'])['m_light'].mean().reset_index()
df_mlight_new



,subject_id,Date,m_light
0,1,2023-08-20,219.763889
1,1,2023-08-21,237.743056
2,1,2023-08-22,161.062500
3,1,2023-08-23,134.826389
4,1,2023-08-24,169.027778
...,...,...,...
101,4,2023-10-27,61.875000
102,4,2023-10-28,83.472222
103,4,2023-10-29,110.131944
104,4,2023-10-30,63.750000


## wHr
스마트워치(갤럭시 워치)에서 측정된 심박 데이터. 1초 간격으로 측정됨.
> **105 rows로 합치는 것 성공**
- subject_id: 실험 참여자의 식별자
- timestamp
- heart_rate

In [ ]:
# 데이터 불러오기
file_name = "ch2024_val__w_heart_rate.parquet.gzip"
df_wHr = pd.read_parquet(os.path.join(challenge2024_dataset_path1, file_name))

# Date 컬럼 새로 만들고 기존의 timestamp 컬럼 제거
df_wHr['Date'] = pd.to_datetime(df_wHr['timestamp'], format='%Y-%m-%d').dt.date
df_wHr.drop(columns = 'timestamp', axis = 1, inplace = True)

# subject_id, Date로 묶어주고 wHr의 하루치  평균을 구해준다
df_wHr_new = df_wHr.groupby(['subject_id', 'Date'])['heart_rate'].mean().reset_index()
df_wHr_new


,subject_id,Date,heart_rate
0,1,2023-08-20,41.247214
1,1,2023-08-21,48.585655
2,1,2023-08-22,44.164511
3,1,2023-08-23,41.837744
4,1,2023-08-24,42.578246
...,...,...,...
100,4,2023-10-27,38.274441
101,4,2023-10-28,40.737421
102,4,2023-10-29,50.755633
103,4,2023-10-30,46.688909


## wPedo
스마트워치(갤럭시 워치)에서 측정된 걸음수 데이터 및 관련 정보. 1분 간격으로 측정됨.
> **105 rows × 8 columns로 합치기 성공**

- subject_id: 실험 참여자의 식별자
- timestamp
- burned_calories: 소모된 칼로리(하루 총량)
- distance: 이동거리(하루 총량)
- running_steps: 뛰어간 걸음 수(하루 총량)
- speed: 속도(평균)
- steps: 분당 걸음 수(평균)
- step_frequency: 걸음 발생 주기(평균)

In [ ]:
# 데이터 불러오기
file_name = "ch2024_val__w_pedo.parquet.gzip"
df_wPedo = pd.read_parquet(os.path.join(challenge2024_dataset_path1, file_name))


# Date 컬럼 새로 만들고 기존의 timestamp 컬럼 제거
df_wPedo['Date'] = pd.to_datetime(df_wPedo['timestamp'], format='%Y-%m-%d').dt.date
df_wPedo.drop(columns = 'timestamp', axis = 1, inplace = True)



df_wPedo_new = df_wPedo.groupby(['subject_id', 'Date'])[['speed', 'steps', 'step_frequency']].mean().reset_index()
df_wPedo_new2 = df_wPedo.groupby(['subject_id', 'Date'])[['burned_calories', 'distance', 'running_steps']].sum().reset_index()
df_wPedo_final = pd.concat([df_wPedo_new, df_wPedo_new2.loc[:, 'burned_calories':]], axis=1)
df_wPedo_final


,subject_id,Date,speed,steps,step_frequency,burned_calories,distance,running_steps
0,1,2023-08-20,1.100692,9.819928,0.163665,382.033447,6252.162109,105
1,1,2023-08-21,0.884221,7.883605,0.131393,547.102295,8544.044922,863
2,1,2023-08-22,1.295501,10.544386,0.175740,386.366943,6238.247188,90
3,1,2023-08-23,1.218778,10.236495,0.170608,391.678711,6465.914062,125
4,1,2023-08-24,0.858235,7.112142,0.118536,1045.159668,6905.455000,91
...,...,...,...,...,...,...,...,...
100,4,2023-10-27,0.541936,4.911251,0.081854,267.439819,5466.734375,241
101,4,2023-10-28,0.605181,4.940124,0.082335,250.587949,4981.944375,332
102,4,2023-10-29,0.415358,3.251935,0.054199,316.766602,2981.113594,169
103,4,2023-10-30,0.724143,5.825832,0.097097,127.102783,2368.691406,312


## wLight
- 스마트워치(갤럭시 워치)에서 빛의 세기. 10분 간격으로 측정됨.
- subject_id: 실험 참여자의 식별자
- timestamp
- w_light: 빛의 세기

In [ ]:
# 데이터 불러오기
file_name = "ch2024_val__w_light.parquet.gzip"
df_wLight = pd.read_parquet(os.path.join(challenge2024_dataset_path1, file_name))


# Date 컬럼 새로 만들고 기존의 timestamp 컬럼 제거
df_wLight['Date'] = pd.to_datetime(df_wLight['timestamp'], format='%Y-%m-%d').dt.date
df_wLight.drop(columns = 'timestamp', axis = 1, inplace = True)

# subject_id, Date로 묶어주고 m_light의 하루치  평균을 구해준다
df_wLight_new = df_wLight.groupby(['subject_id', 'Date'])['w_light'].mean().reset_index()
df_wLight_new


,subject_id,Date,w_light
0,1,2023-08-20,1152.152778
1,1,2023-08-21,903.868056
2,1,2023-08-22,674.618056
3,1,2023-08-23,1097.562500
4,1,2023-08-24,417.748252
...,...,...,...
100,4,2023-10-27,230.284722
101,4,2023-10-28,53.635714
102,4,2023-10-29,72.654412
103,4,2023-10-30,111.351351


## gps에 빠진 4개의 row를 알아보자
**55, 79, 83, 84 row에 gps관련 정보가 nan**


In [ ]:
missing_value = df_mact_fin.merge(df_mgps_new, on=['subject_id', 'Date'], how='left') # macc, wlight
missing_value
nan_rows = missing_value[missing_value.isna().any(axis=1)]
nan_rows # 55, 79, 83, 84 row에 gps관련 정보가 nan

,subject_id,Date,m_activity,IN_VEHICLE,ON_BICYCLE,STILL,UNKNOWN,WALKING,RUNNING,altitude,latitude,longitude,speed
55,2,2023-09-30,1440,69,0,688,582,94,7,NaN,NaN,NaN,NaN
79,4,2023-10-06,1440,99,0,1314,0,27,0,NaN,NaN,NaN,NaN
83,4,2023-10-10,1439,68,0,1352,1,18,0,NaN,NaN,NaN,NaN
84,4,2023-10-11,1440,145,0,1282,0,13,0,NaN,NaN,NaN,NaN


In [ ]:
total = df_mact_fin.merge(df_wLight_new, on=['subject_id', 'Date'], how='inner') # macc, wlight
total = total.merge(df_wHr_new, on=['subject_id', 'Date'], how='inner') # df_wHr_new
total = total.merge(df_wPedo_final, on=['subject_id', 'Date'], how='inner') # df_wPedo_final
total = total.merge(df_mlight_new, on=['subject_id', 'Date'], how='inner') # df_mlight_new
total = total.merge(df_mgps_new, on=['subject_id', 'Date'], how='left') # df_mgps_new
total = total.merge(sound_count, on=['subject_id', 'Date'], how='left') # df_mgps_new
total

,subject_id,Date,m_activity,IN_VEHICLE,ON_BICYCLE,STILL,UNKNOWN,WALKING,RUNNING,w_light,...,White noise,"Whoosh, swoosh, swish",Wild animals,Wind,Wind chime,Wind noise (microphone),Wood,Writing,Zipper (clothing),unknown
0,1,2023-08-20,1440,105,17,1153,0,150,15,1152.152778,...,6,0,0,2,0,0,1,1,0,0
1,1,2023-08-21,1440,92,25,1061,0,256,6,903.868056,...,41,0,0,5,0,0,0,0,1,0
2,1,2023-08-22,1440,105,48,869,0,402,16,674.618056,...,30,0,1,0,0,0,0,0,0,0
3,1,2023-08-23,1440,133,51,963,0,293,0,1097.562500,...,6,0,0,3,0,0,0,0,0,0
4,1,2023-08-24,1440,141,10,1069,0,215,5,417.748252,...,9,0,0,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,4,2023-10-27,1431,73,0,1336,1,21,0,230.284722,...,4,0,1,0,0,0,0,0,0,0
101,4,2023-10-28,1440,47,0,1385,0,8,0,53.635714,...,9,0,0,5,0,0,1,1,0,0
102,4,2023-10-29,1440,121,0,1303,0,16,0,72.654412,...,17,0,1,1,0,1,0,0,0,0
103,4,2023-10-30,1440,60,0,1369,0,11,0,111.351351,...,0,0,0,2,0,0,0,0,0,0


## 레이블 파일과 합치기

In [ ]:
file_name = "val_label.csv"
df_label = pd.read_csv(os.path.join(challenge2024_dataset_path, file_name))
## df_label
total.rename(columns={'Date': 'date'}, inplace=True)
total['date'] = total['date'].astype(str)
result = total.merge(df_label, on=['subject_id', 'date'], how='inner')
result

,subject_id,date,m_activity,IN_VEHICLE,ON_BICYCLE,STILL,UNKNOWN,WALKING,RUNNING,w_light,...,Writing,Zipper (clothing),unknown,Q1,Q2,Q3,S1,S2,S3,S4
0,1,2023-08-20,1440,105,17,1153,0,150,15,1152.152778,...,1,0,0,1,1,1,0,0,0,0
1,1,2023-08-21,1440,92,25,1061,0,256,6,903.868056,...,0,1,0,1,1,1,0,0,1,0
2,1,2023-08-22,1440,105,48,869,0,402,16,674.618056,...,0,0,0,0,1,1,0,1,1,0
3,1,2023-08-23,1440,133,51,963,0,293,0,1097.562500,...,0,0,0,0,1,1,0,0,1,0
4,1,2023-08-24,1440,141,10,1069,0,215,5,417.748252,...,0,0,0,1,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,4,2023-10-27,1431,73,0,1336,1,21,0,230.284722,...,0,0,0,0,1,0,0,1,1,1
101,4,2023-10-28,1440,47,0,1385,0,8,0,53.635714,...,1,0,0,1,1,0,1,1,1,1
102,4,2023-10-29,1440,121,0,1303,0,16,0,72.654412,...,0,0,0,1,1,0,0,1,1,1
103,4,2023-10-30,1440,60,0,1369,0,11,0,111.351351,...,0,0,0,0,1,0,0,0,1,1
